In [ ]:
'''
    Special tokens: removed from steps
'''

In [1]:
# Sets seeds for reproducability.

import torch
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

In [2]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import *
from accuracy import *
from transformers import get_linear_schedule_with_warmup
from model import Model
from torch import nn

import numpy as np
import torch
import matplotlib.pyplot as plt

def train(model, num_actions_train=8, num_actions_valid=6, batch_size=4, epochs=25, lr=0.001, MAX_DETECTIONS=20):
    num_actions = num_actions_train
    
    train_dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    valid_dataset = YouCookII(num_actions_valid, "/h/sagar/ece496-capstone/datasets/ycii")
    
    train_size = len(train_dataset)
    valid_size = len(valid_dataset)
    
    print("Training Dataset Size: {}, Validation Dataset Size: {}".format(train_size, valid_size))
        
    collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True, worker_init_fn=seed_worker)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True, worker_init_fn=seed_worker)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(0.2*epochs), epochs)

    train_loss = np.zeros(epochs)
    valid_loss = np.zeros(epochs)
    
    train_accuracy = np.zeros(epochs)
    valid_accuracy = np.zeros(epochs)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_loss = 0.
        num_batches = 0
        
        total = 0
        correct = 0
                
        for data in train_dataloader:
            _, bboxes, features, steps, entities, entity_count, _, _ = data
            
            # Zero out any gradients.
            optimizer.zero_grad()
            
            # Run inference (forward pass).
            loss_data, VG, RR = model(steps, features, bboxes, entities, entity_count)
            
            # Loss from alignment.
            loss_ = compute_loss_batched(loss_data)

            # Backpropagation (backward pass).
            loss_.backward()

            # Update parameters.
            optimizer.step()
            
            epoch_loss += loss_
            num_batches += 1
            
        # Scheduler update.
        scheduler.step()
        epoch_loss = epoch_loss / (num_batches * batch_size)
        
        # Save loss and accuracy at each epoch and plot.
        train_loss[epoch] = float(epoch_loss)
        train_accuracy[epoch] = get_alignment_accuracy(model, train_dataloader, batch_size, num_actions) 
        
        valid_loss[epoch] = get_alignment_loss(model, valid_dataloader, batch_size, num_actions)
        valid_accuracy[epoch] = get_alignment_accuracy(model, valid_dataloader, batch_size, num_actions)

        print("Epoch {} - Train Loss: {:.2f}, Validation Loss: {:.2f}, Train Accuracy: {:.2f}, Validation Accuracy: {:.2f}"
              .format(epoch + 1, train_loss[epoch], valid_loss[epoch], train_accuracy[epoch], valid_accuracy[epoch]))
    
    plt.figure()
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.legend()
    
    plt.figure()
    plt.plot(train_accuracy, label='train accuracy')
    plt.plot(valid_accuracy, label='valid accuracy')
    plt.legend()
    
    plt.show()
        
    return train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(device)

In [ ]:
# Trainer.

num_actions_train = 8
num_actions_valid = 6
batch_size = 4
epochs = 100
lr = 1e-4

train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data = train(
    model, 
    num_actions_train=num_actions_train, 
    num_actions_valid=num_actions_valid, 
    batch_size=batch_size, 
    epochs=epochs, 
    lr=lr
)

In [4]:
# Evaluation.

from eval_fi import eval_all_dataset
eval_all_dataset(model, path="/h/sagar/ece496-capstone/datasets/fi")

--------------------------------------------------
Action 1: in a bowl, add ground beef, salt, pepper, 1 egg, dijon mustard and bread crumbs
Best IoU possible = 0.8062977194786072
Random Candidate IoU = 0.5997998118400574
Chosen Frame IoU: 0.4818645417690277
Best IoU possible = 0.5932643413543701
Random Candidate IoU = 0.43438395857810974
Chosen Frame IoU: 0.37369081377983093
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: mix the ingredients with hand
Best IoU possible = 0.6298992037773132
Random Candidate IoU = 0.35797545313835144
Chosen Frame IoU: 0.3041292726993561
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add vegetable oil in a pan
Best IoU possible = 0.7165105938911438
Random Candida

--------------------------------------------------
Action 1: cut the cabbage into small pieces
Best IoU possible = 0.9013723731040955
Random Candidate IoU = 0.15885815759608485
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: Add carawat seeds, sea salt
Best IoU possible = 0.11877390742301941
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.03214755344321466
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: mix them evenly in the cabbage
Best IoU possible = 0.22755128145217896
Random Candidate IoU = 0.10723605006933212
Chosen Frame IoU: 0.0
Best IoU possible = 0.9315666928471509
Random Candidate IoU = 0.054621756076812744
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 4: Add some leftover brine in a jar
Best IoU possible = 0.23314441740512848
Random Candidate IoU = 0.07153001427650452
Chosen Frame IoU: 0.083778977394104
Be

Best IoU possible = 0.6718937158584595
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.7263962626457214
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 9: Pour the marinade over the ribs
Best IoU possible = 0.793299674987793
Random Candidate IoU = 0.38696542382240295
Chosen Frame IoU: 0.08354558795690536
Best IoU possible = 0.7041696906089783
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.04399796947836876
Mean Upper Bound IoU: 0.542595624923706, Mean Random IoU: 0.06744249910116196, Mean Model IoU: 0.04771680012345314
Top-1 acc@0.5:
Proposal Upper Bound: 0.6666666666666666, Random: 0.0, Model: 0.0
--------------------------------------------------
Action 1: boil water in a pot with salt and oil
Best IoU possible = 0.6374806761741638
Random Candidate IoU = 0.3594052791595459
Chosen Frame IoU: 0.49588990211486816
Best IoU possible = 0.5912726521492004
Random Candidate IoU = 0.4232030212879181
Chosen Frame 

Best IoU possible = 0.30627989768981934
Random Candidate IoU = 0.19799844920635223
Chosen Frame IoU: 0.3046085834503174
--------------------------------------------------
Action 8: add ham, shredded cheese and more sauce and ham
Best IoU possible = 0.7701946496963501
Random Candidate IoU = 0.6828290820121765
Chosen Frame IoU: 0.4797215163707733
Best IoU possible = 0.6028886437416077
Random Candidate IoU = 0.5467394590377808
Chosen Frame IoU: 0.168413445353508
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 9: Cover it with another piece of bread
Best IoU possible = 0.7562223672866821
Random Candidate IoU = 0.24228504300117493
Chosen Frame IoU: 0.33420929312705994
Best IoU possible = 0.5972216129302979
Random Candidate IoU = 0.1085890606045723
Chosen Frame IoU: 0.22125348448753357
--------------------------------------------------
Action 10: add more sauce on top
Best IoU possible = 0.524

--------------------------------------------------
Action 1: Cook pancetta in a pan with oil
Best IoU possible = 0.6409754157066345
Random Candidate IoU = 0.0041260248981416225
Chosen Frame IoU: 0.3682132661342621
Best IoU possible = 0.769189715385437
Random Candidate IoU = 0.036373961716890335
Chosen Frame IoU: 0.28497329354286194
--------------------------------------------------
Action 2: Melt butter in a pan
Best IoU possible = 0.5938389301300049
Random Candidate IoU = 0.39860716462135315
Chosen Frame IoU: 0.24712221324443817
Best IoU possible = 0.5649166703224182
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 3: mix the flour with the butter
Best IoU possible = 0.5315556078534311
Random Candidate IoU = 0.3286047577857971
Chosen Frame IoU: 0.0
Best IoU possible = 0.528659110126503
Random Candidate IoU = 0.23465509712696075
Chosen Frame IoU: 0.1848941445350647
--------------------------------------------------
Action 4: Sti

...

Best IoU possible = 0.7422332167625427
Random Candidate IoU = 0.1572566032409668
Chosen Frame IoU: 0.2968854606151581
Best IoU possible = 0.795101523399353
Random Candidate IoU = 0.5722202658653259
Chosen Frame IoU: 0.37377139925956726
--------------------------------------------------
Action 21: Spread melted butter on a plate
Best IoU possible = 0.0515613928437233
Random Candidate IoU = 0.01389435420429256
Chosen Frame IoU: 0.05046134814620018
Best IoU possible = 0.044203195720911026
Random Candidate IoU = 0.0156596340239048
Chosen Frame IoU: 0.008308649063110352
--------------------------------------------------
Action 22: transfer the cooked pierogies
Best IoU possible = 0.022247500717639923
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0037605040706694126
Mean Upper Bound IoU: 0.3813657760620117, Mean Random IoU: 0.13882319629192352, Mean Model IoU: 0.14588604867458344
Top-1 acc@0.5:
Proposal Upper Bound: 0.2962962962962963, Random: 0.1111111111111111, Model: 0.0
----------------

Best IoU possible = 0.3749147653579712
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.12691381874167076
Best IoU possible = 0.0777507871389389
Random Candidate IoU = 0.023362910374999046
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 11: place the seasoned foie gras in the pan
Best IoU possible = 0.8046154975891113
Random Candidate IoU = 0.18394346990347313
Chosen Frame IoU: 0.8046154975891113
Best IoU possible = 0.7191280126571655
Random Candidate IoU = 0.355810284614563
Chosen Frame IoU: 0.022995473816990852
--------------------------------------------------
Action 12: cook until it is golden brown
No entities detected for this action.
--------------------------------------------------
Action 13: Remove the cooked foie gras
Best IoU possible = 0.7091792821884155
Random Candidate IoU = 0.12166352570056915
Chosen Frame IoU: 0.1075330376625061
--------------------------------------------------
Action 14: pat i

--------------------------------------------------
Action 1: spread rice onto the seaweed
Best IoU possible = 0.1831646820340744
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.1831646820340744
Best IoU possible = 0.5116598606109619
Random Candidate IoU = 0.12955191731452942
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 2: sprinkle sesame seeds on top of the rice
Best IoU possible = 0.21053092357137176
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.7744829285521743
Random Candidate IoU = 0.13378492859169425
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 3: Flip the sheet over
Best IoU possible = 0.26849167538054064
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.013397102876502956
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: place tuna in a line on the sheet
Best IoU possible = 0.1698647141456604
Random Candidate IoU = 0.05789216235280037

Best IoU possible = 0.5912720561027527
Random Candidate IoU = 0.4550965130329132
Chosen Frame IoU: 0.4550965130329132
Best IoU possible = 0.7910395860671997
Random Candidate IoU = 0.5348484516143799
Chosen Frame IoU: 0.3098036050796509
--------------------------------------------------
Action 8: Chop some lemon and lime
Best IoU possible = 0.8422186970710754
Random Candidate IoU = 0.6921311020851135
Chosen Frame IoU: 0.7701495885848999
This entity has no ground truth bounding box
Mean Upper Bound IoU: 0.5891347527503967, Mean Random IoU: 0.2723270356655121, Mean Model IoU: 0.3611772656440735
Top-1 acc@0.5:
Proposal Upper Bound: 0.8181818181818182, Random: 0.18181818181818182, Model: 0.18181818181818182
--------------------------------------------------
Action 1: mix lemon juice and milk
Best IoU possible = 0.544775128364563
Random Candidate IoU = 0.14913614094257355
Chosen Frame IoU: 0.06459014117717743
This entity has no ground truth bounding box
--------------------------------------

Best IoU possible = 0.7623729109764099
Random Candidate IoU = 0.12551027536392212
Chosen Frame IoU: 0.39609917998313904
Mean Upper Bound IoU: 0.4630141258239746, Mean Random IoU: 0.11192569881677628, Mean Model IoU: 0.08469759672880173
Top-1 acc@0.5:
Proposal Upper Bound: 0.5833333333333334, Random: 0.08333333333333333, Model: 0.0
--------------------------------------------------
EVALUATION SUMMARY
Number of videos: 54
Mean IoU:
	Proposal Upper Bound: 0.5761603116989136
	Random: 0.17854449152946472
	Model: 0.2056303769350052
Top-1 accuracy@0.5:
	Proposal Upper Bound: 70.1%
	Random: 8.1%
	Model: 10.7%
--------------------------------------------------


In [ ]:
# Visualizer.

from visualizer import inference

YCII = "/h/sagar/ece496-capstone/datasets/ycii"
FI = "/h/sagar/ece496-capstone/datasets/fi"

VG, RR = inference(model, 27, 0, FI)

In [ ]:
# Saving and loading weights.

SAVE = False
LOAD = True

if SAVE:
    torch.save(model.state_dict(), "/h/sagar/ece496-capstone/weights/t1")
    
if LOAD:
    model.load_state_dict(torch.load("/h/sagar/ece496-capstone/weights/t1"))

In [ ]:
# Reload modules.

import importlib
import visualizer
import eval_fi
import model as mdl
import loss

importlib.reload(visualizer)
importlib.reload(eval_fi)
importlib.reload(mdl)
importlib.reload(loss)